### Importing essential libraries 

In [1]:
import pandas as pd 
import numpy as np 
import requests 
from bs4 import BeautifulSoup
from tqdm import tqdm

link = 'https://github.com/AshishJangra27'

soup = BeautifulSoup(requests.get(link).content,'html.parser')

### Defining a social account functions 

In [2]:
def social_account(urls):
    accounts = []
    for url in urls:
        if 'kaggle' in url:
            accounts.append('Kaggle')
        elif 'instagram' in url:
            accounts.append('Instagram')
        elif 'facebook' in url:
            accounts.append('Facebook')
        elif 'linkedin' in url:
            accounts.append('Linkedin')
        elif 'github' in url:
            accounts.append('Github')
        else:
            accounts.append('Personal website')
    return ','.join(accounts)

### Scraping the data of the github repositories 

In [3]:
user_name  = soup.find('span',class_ = 'p-name vcard-fullname d-block overflow-hidden').text.strip()
user_id    = soup.find('span',class_ = 'p-nickname vcard-username d-block').text.strip()
bio        = soup.find('div',class_ = 'js-profile-editable-area d-flex flex-column d-md-block').find('div').text
lives_in   = soup.find('li',class_ = 'vcard-detail pt-1 hide-sm hide-md').text.strip()
data = []

for i in soup.find_all('a',class_ = 'Link--secondary no-underline no-wrap'):
    if 'follower' in (i.text):
        follower = int(i.find('span').text.strip())
    elif 'following' in i.text:
        following = int(i.find('span').text.strip())
    try:        
        content  = [int(content.find('span').text.strip()) for content in soup.find('nav',class_ ='UnderlineNav-body width-full p-responsive js-sidenav-container-pjax').find_all('a')[1:]]
        repos    = content[0]
        project  = content[1]
        packages = content[2]
        stars    = content[3]
    except:
        repos    = np.nan
        project  = np.nan
        packages = np.nan
        stars    = np.nan
    try:
        url = [links.find('a')['href'] for links in soup.find_all('li',class_='vcard-detail pt-1')]
        account = social_account(url)
    except:
        url = np.nan
        account = 'no social account has been linked'


data.append([user_id , user_name , bio , follower , following ,lives_in,url,account, repos , project , packages , stars])

df = pd.DataFrame(data,columns = ['user_id' ,'user_name' ,'bio' ,'follower' ,'following', 'lives_in', 'url','account', 'repos' , 'project' , 'packages' , 'stars'])
df.head()




,user_id,user_name,bio,follower,following,lives_in,url,account,repos,project,packages,stars
0,AshishJangra27,Ashish Jangra,I am an AI Trainer. \nI train students on tech...,305,1,Noida,[],,76,0,0,52


### Scraping the repos

In [4]:
link = 'https://github.com/AshishJangra27?tab=repositories'
repos_links = []
repos_name  = []
for pages in range(int(round(df['repos']/30))):
    res = requests.get(link)
    soup = BeautifulSoup(res.content,'html.parser')
    for name in soup.find_all('div',class_ ='d-inline-block mb-1'):
        repos_links.append('https://www.github.com' + name.find('a').get('href'))
    for name in soup.find_all('div',class_ ='d-inline-block mb-1'):
        repos_name.append(name.find('a').text.strip())
    try:
        link = 'https://www.github.com' + soup.find('div',{'class':'paginate-container'}).find('a',class_ = 'next_page').get('href')    
    except:
        pass

### Scraping the stars

In [5]:
link = 'https://github.com/AshishJangra27?tab=stars'

stars_repo_link = []
stars_repo_name = []

for pages in range(int(round(df['stars']/30))):
    res = requests.get(link).text
    soup = BeautifulSoup(res,'html.parser')
    for stars in soup.find_all('div',class_ = 'd-inline-block mb-1'):
        stars_repo_link.append('https://www.github.com' + stars.find('a').get('href'))
    for stars in soup.find_all('div',class_ = 'd-inline-block mb-1'):
        stars_repo_name.append(stars.find('a').text.strip().split('/')[1].strip())
    try:
        link = soup.find('div',{'class':'paginate-container'}).find('a',class_ = 'next_page').get('href')
    except:
        pass

### Concating the data 


In [6]:
df_ = pd.DataFrame()

df_['repos_name']       = [repos_name]
df_['repos_links']      = [repos_links]
df_['stars_repos']      = [stars_repo_link]
df_['stars_repos_link'] = [stars_repo_name]

df = pd.concat([df,df_],axis = 1)
df

,user_id,user_name,bio,follower,following,lives_in,url,account,repos,project,packages,stars,repos_name,repos_links,stars_repos,stars_repos_link
0,AshishJangra27,Ashish Jangra,I am an AI Trainer. \nI train students on tech...,305,1,Noida,[],,76,0,0,52,"[In-One-Go, Data-Science-Live-GFG, Airbnb-Scra...",[https://www.github.com/AshishJangra27/In-One-...,[https://www.github.com/AshishJangra27/Gaana-D...,"[Gaana-Data-Analysis, GFG-Tableau-Course-Datas..."


### Scraping the followers

In [7]:
user_id_follower  = []
follower_username = []
for pages in range(1,int(round(df['follower']/50))+1):
    link = 'https://github.com/AshishJangra27?page=' + str(pages) + '&tab=followers'   # scraping all the followers
    res = requests.get(link).text
    soup = BeautifulSoup(res,'html.parser')
    for followers in soup.find_all('div',class_ = 'd-table table-fixed col-12 width-full py-4 border-bottom color-border-muted'):
        user_id_follower.append(followers.find_all('span')[1].text)                    # getting the user id of the follower
    for name in soup.find_all('div',class_ = 'd-table table-fixed col-12 width-full py-4 border-bottom color-border-muted'):
        follower_username.append(name.find_all('span')[0].text.strip())                # getting the username of the follower

In [8]:
followers = pd.DataFrame({'user_id':user_id_follower,'username':follower_username})
followers

,user_id,username
0,ankitpanda008,Ankit Kumar Panda
1,MohanPrathap,
2,imayank45,Mayank Kathane
3,ChidambaraRaju,Chidambara Raju
4,prashant-3200,
...,...,...
295,MadhurRamachandruni,Madhur Ramachandruni
296,Arnav1145,ARNAV MODANWAL
297,charishmaseera,Charishma Seera
298,shameerjs93,


# Data of the followers 

In [10]:
data = []

for i in tqdm(followers['user_id']):
    link = 'https://www.github.com/'+i
    soup = BeautifulSoup(requests.get(link).content,'html.parser')
    user_name  = soup.find('span',class_ = 'p-name vcard-fullname d-block overflow-hidden').text.strip()
    user_id    = soup.find('span',class_ = 'p-nickname vcard-username d-block').text.strip()
    try:
        bio        = soup.find('div',class_ = 'js-profile-editable-area d-flex flex-column d-md-block').find('div').text
    except:
        bio        = 'No bio written'
    try:
        lives_in   = soup.find('li',class_ = 'vcard-detail pt-1 hide-sm hide-md').text.strip()
    except:
        lives_in   = np.nan

    for i in soup.find_all('a',class_ = 'Link--secondary no-underline no-wrap'):
        if 'follower' in (i.text):
            follower = i.find('span').text.strip()
        elif 'following' in i.text:
            following = i.find('span').text.strip()
        try:        
            content  = [int(content.find('span').text.strip()) for content in soup.find('nav',class_ ='UnderlineNav-body width-full p-responsive js-sidenav-container-pjax').find_all('a')[1:]]
            repos    = content[0]
            project  = content[1]
            packages = content[2]
            stars    = content[3]
        except:
            repos    = np.nan
            project  = np.nan
            packages = np.nan
            stars    = np.nan
        try:
            url = [links.find('a')['href'] for links in soup.find_all('li',class_='vcard-detail pt-1')]
            account = social_account(url)
        except:
            url = np.nan
            account = 'no social account has been linked'


    data.append([user_id , user_name , bio , follower , following ,lives_in,url,account, repos , project , packages , stars])

followers_data = pd.DataFrame(data,columns = ['user_id' ,'user_name' ,'bio' ,'follower' ,'following', 'lives_in', 'url','account', 'repos' , 'project' , 'packages' , 'stars'])
followers_data.to_csv('followers_.csv',index = False)

100%|█████████████████████████████████████████| 300/300 [08:26<00:00,  1.69s/it]


# Saving the followers_data

In [11]:
followers_data = pd.read_csv('followers_.csv')
followers_data.head()

,user_id,user_name,bio,follower,following,lives_in,url,account,repos,project,packages,stars
0,ankitpanda008,Ankit Kumar Panda,Data Science enthusiast.,0,1,NaN,[],NaN,3.0,0.0,0.0,0.0
1,MohanPrathap,NaN,NaN,0,2,NaN,[],NaN,0.0,0.0,0.0,0.0
2,imayank45,Mayank Kathane,NaN,0,1,NaN,[],NaN,22.0,0.0,0.0,3.0
3,ChidambaraRaju,Chidambara Raju,A Data Alchemist who Transforms Raw Data into ...,4,6,NaN,['https://www.kaggle.com/rajugc'],Kaggle,14.0,0.0,0.0,34.0
4,prashant-3200,NaN,NaN,0,1,NaN,[],NaN,0.0,0.0,0.0,0.0


## Pre-processing the follower_data

In [12]:
followers_data['account']   = followers_data['account'].fillna('no social account has been linked')
followers_data['bio']       = followers_data['bio'].fillna('No bio')
followers_data['lives_in']  = followers_data['lives_in'].fillna('no data')
followers_data['url']       = followers_data['url'].fillna('no url')
followers_data['user_name'] = followers_data['user_name'].fillna('No username')
followers_data['repos']     = followers_data['repos'].fillna(0.0)
followers_data['packages']  = followers_data['packages'].fillna(0.0)
followers_data['project']   = followers_data['project'].fillna(0.0)
followers_data['stars']     = followers_data['stars'].fillna(0.0)

### Creating pages from total repositories and total stars repositories

In [13]:
pages     = []
star_page = []

# repos pages 
for i in followers_data['repos']:
    pages.append(round(int(i)/30))
    
# star pages 
for j in followers_data['stars']:
    star_page.append(round(int(j)/30))

# Adding the values to the follower_data
followers_data['repo_pages'] = pages
followers_data['star_pages'] = star_page

In [14]:
followers_data.head()

,user_id,user_name,bio,follower,following,lives_in,url,account,repos,project,packages,stars,repo_pages,star_pages
0,ankitpanda008,Ankit Kumar Panda,Data Science enthusiast.,0,1,no data,[],no social account has been linked,3.0,0.0,0.0,0.0,0,0
1,MohanPrathap,No username,No bio,0,2,no data,[],no social account has been linked,0.0,0.0,0.0,0.0,0,0
2,imayank45,Mayank Kathane,No bio,0,1,no data,[],no social account has been linked,22.0,0.0,0.0,3.0,1,0
3,ChidambaraRaju,Chidambara Raju,A Data Alchemist who Transforms Raw Data into ...,4,6,no data,['https://www.kaggle.com/rajugc'],Kaggle,14.0,0.0,0.0,34.0,0,1
4,prashant-3200,No username,No bio,0,1,no data,[],no social account has been linked,0.0,0.0,0.0,0.0,0,0


## Scraping the Repository data of all the followers 

In [15]:
data = []
for users in tqdm(followers_data.values):

    link_ = 'https://github.com/' + str(users[0]) + '?tab=repositories'
    repos_links = []
    repos_name  = []
    if users[12]>0:
        for pages in range(int(users[12])):
            res = requests.get(link_).text
            soup = BeautifulSoup(res,'html.parser')
            for name in soup.find_all('div',class_ ='d-inline-block mb-1'):
                repos_links.append('https://www.github.com' + name.find('a').get('href'))
            for name in soup.find_all('div',class_ ='d-inline-block mb-1'):
                repos_name.append(name.find('a').text.strip())
            try:
                link = 'https://www.github.com' + soup.find('div',{'class':'paginate-container'}).find('a',class_ = 'next_page').get('href')    
            except:
                pass
    else:
        for pages in range(int(users[12]+1)):
            res = requests.get(link_).text
            soup = BeautifulSoup(res,'html.parser')
            for name in soup.find_all('div',class_ ='d-inline-block mb-1'):
                repos_links.append('https://www.github.com' + name.find('a').get('href'))
            for name in soup.find_all('div',class_ ='d-inline-block mb-1'):
                repos_name.append(name.find('a').text.strip())
            try:
                link = 'https://www.github.com' + soup.find('div',{'class':'paginate-container'}).find('a',class_ = 'next_page').get('href')    
            except:
                pass
    data.append([repos_name,repos_links])

100%|█████████████████████████████████████████| 300/300 [07:12<00:00,  1.44s/it]


### Saving the repository data in a file

In [16]:
with open('repos_data.txt','w') as fd:
    fd.write(str(data))
    fd.close()

## Stars scraper for the followers ( stars data something is missing stars data is wrong)

In [17]:
data1 = []

for users in tqdm(followers_data.values):

    link_ = 'https://github.com/' + users[0] + '?tab=stars'
    stars_repo_link = []
    stars_repo_name = []
    
    if users[13]>0:
        for pages in range(users[13]):
            res = requests.get(link_).text
            soup = BeautifulSoup(res,'html.parser')
            for stars in soup.find_all('div',class_ = 'd-inline-block mb-1'):
                stars_repo_link.append('https://www.github.com' + stars.find('a').get('href'))
            for stars in soup.find_all('div',class_ = 'd-inline-block mb-1'):
                stars_repo_name.append(stars.find('a').text.strip())
            try:
                link_ = soup.find('div',{'class':'paginate-container'}).find('a',class_ = 'next_page').get('href')
            except:
                pass
    else:
        for pages in range(users[12]+1):
            res = requests.get(link_).text
            soup = BeautifulSoup(res,'html.parser')
            for stars in soup.find_all('div',class_ = 'd-inline-block mb-1'):
                stars_repo_link.append('https://www.github.com' + stars.find('a').get('href'))
            for stars in soup.find_all('div',class_ = 'd-inline-block mb-1'):
                stars_repo_name.append(stars.find('a').text.strip())
            try:
                link_ = soup.find('div',{'class':'paginate-container'}).find('a',class_ = 'next_page').get('href')
            except:
                pass
            
    data1.append([stars_repo_link,stars_repo_name])
    
    

100%|█████████████████████████████████████████| 300/300 [06:12<00:00,  1.24s/it]


### Saving the Stars data in a file

In [18]:
with open('stars_data.txt','w') as fd:
    fd.write(str(data1))
    fd.close()

## Reading the repository and stars files

In [19]:
# reading the repos file 

with open('repos_data.txt','r') as nfd:
    t = nfd.read()
    nfd.close()
    
repo_data_ = eval(t)                       # converting the string to list/tuple

# reading the stars files

with open('stars_data.txt','r') as fd:
    text = fd.read()
    fd.close()
    
data_ = eval(text)                        # converting the string to list/tuple

## Adding the star name, links 

In [20]:
# star repository and name
star_links_ = []
star_names_ = []

# repository name and links
repo_names_  = []  
repos_links_ = []

for i in range(len(data_)):
    star_names_.append(data_[i][1])
    star_links_.append(data_[i][0])

for j in range(len(repo_data_)):
    repo_names_.append(repo_data_[j][0])
    repos_links_.append(repo_data_[j][1])

followers_data['repos_name'] = repo_names_
followers_data['repos_link'] = repos_links_
followers_data['stars_repos'] = star_names_
followers_data['stars_repos_links'] = star_links_

In [21]:
followers_data.head()

,user_id,user_name,bio,follower,following,lives_in,url,account,repos,project,packages,stars,repo_pages,star_pages,repos_name,repos_link,stars_repos,stars_repos_links
0,ankitpanda008,Ankit Kumar Panda,Data Science enthusiast.,0,1,no data,[],no social account has been linked,3.0,0.0,0.0,0.0,0,0,"[CV, Projects-Data-Analysis, linkedin-skill-as...","[https://www.github.com/ankitpanda008/CV, http...",[],[]
1,MohanPrathap,No username,No bio,0,2,no data,[],no social account has been linked,0.0,0.0,0.0,0.0,0,0,[],[],[],[]
2,imayank45,Mayank Kathane,No bio,0,1,no data,[],no social account has been linked,22.0,0.0,0.0,3.0,1,0,"[Data-Analysis, SQL-MySQL, GDP-Analysis, Stock...",[https://www.github.com/imayank45/Data-Analysi...,"[imayank45 / Data-Preprocessing, imayank45 / O...",[https://www.github.com/imayank45/Data-Preproc...
3,ChidambaraRaju,Chidambara Raju,A Data Alchemist who Transforms Raw Data into ...,4,6,no data,['https://www.kaggle.com/rajugc'],Kaggle,14.0,0.0,0.0,34.0,0,1,"[Web-Scraping-Project-IPL-Dataset, Youtube-Dat...",[https://www.github.com/ChidambaraRaju/Web-Scr...,[ChidambaraRaju / Web-Scraping-Project-IPL-Dat...,[https://www.github.com/ChidambaraRaju/Web-Scr...
4,prashant-3200,No username,No bio,0,1,no data,[],no social account has been linked,0.0,0.0,0.0,0.0,0,0,[],[],[],[]


# pre_processing 
### 1. url 

In [22]:
url = []
for i in followers_data['url']:
    try:
        if len(eval(i)) > 0:
            url.append(','.join(eval(i)))
        else:
            url.append('no url')
    except:
        url.append(i)
        
followers_data['url'] = url

### 2. repos

In [23]:
repos = []
for i in followers_data['repos']:
    repos.append(int(i))

followers_data['repos'] = repos

### 3. project

In [24]:
repos = []
for i in followers_data['repos']:
    repos.append(int(i))

followers_data['repos'] = repos

### 4. stars 

In [25]:
stars = []
for i in followers_data['stars']:
    stars.append(int(i))
    
followers_data['stars'] = stars

### 5.repos_name

In [26]:
repos_name = []
for i in followers_data['repos_name']:
    if len(i)>0:
        repos_name.append(','.join(i))
    else:
        repos_name.append('No Repository')
        
followers_data['repos_name'] = repos_name

### 6.repos_link

In [27]:
repos_link = []
for i in followers_data['repos_link']:
    if len(i)>0:
        repos_link.append(','.join(i))
    else:
        repos_link.append('No Repository link')
        
followers_data['repos_link'] = repos_link

### 7. stars_repos

In [28]:
stars_repos = []
for i in followers_data['stars_repos']:
    if len(i)>0:
        stars_repos.append(','.join(i))
    else:
        stars_repos.append('No Repository link')
        
followers_data['stars_repos'] = stars_repos

### 8. stars_repos_links

In [29]:
stars_repos_links = []
for i in followers_data['stars_repos_links']:
    if len(i)>0:
        stars_repos_links.append(','.join(i))
    else:
        stars_repos_links.append('No Repository link')
        
followers_data['stars_repos_links'] = stars_repos_links
followers_data.head()

,user_id,user_name,bio,follower,following,lives_in,url,account,repos,project,packages,stars,repo_pages,star_pages,repos_name,repos_link,stars_repos,stars_repos_links
0,ankitpanda008,Ankit Kumar Panda,Data Science enthusiast.,0,1,no data,no url,no social account has been linked,3,0.0,0.0,0,0,0,"CV,Projects-Data-Analysis,linkedin-skill-asses...","https://www.github.com/ankitpanda008/CV,https:...",No Repository link,No Repository link
1,MohanPrathap,No username,No bio,0,2,no data,no url,no social account has been linked,0,0.0,0.0,0,0,0,No Repository,No Repository link,No Repository link,No Repository link
2,imayank45,Mayank Kathane,No bio,0,1,no data,no url,no social account has been linked,22,0.0,0.0,3,1,0,"Data-Analysis,SQL-MySQL,GDP-Analysis,Stock-Dat...",https://www.github.com/imayank45/Data-Analysis...,"imayank45 / Data-Preprocessing,imayank45 / OS-...",https://www.github.com/imayank45/Data-Preproce...
3,ChidambaraRaju,Chidambara Raju,A Data Alchemist who Transforms Raw Data into ...,4,6,no data,https://www.kaggle.com/rajugc,Kaggle,14,0.0,0.0,34,0,1,"Web-Scraping-Project-IPL-Dataset,Youtube-Data-...",https://www.github.com/ChidambaraRaju/Web-Scra...,ChidambaraRaju / Web-Scraping-Project-IPL-Data...,https://www.github.com/ChidambaraRaju/Web-Scra...
4,prashant-3200,No username,No bio,0,1,no data,no url,no social account has been linked,0,0.0,0.0,0,0,0,No Repository,No Repository link,No Repository link,No Repository link


## Deleting the unneccessary column 

In [30]:
del followers_data['project']
del followers_data['repo_pages']
del followers_data['star_pages']

In [33]:
followers_data.head()

,user_id,user_name,bio,follower,following,lives_in,url,account,repos,packages,stars,repos_name,repos_link,stars_repos,stars_repos_links
0,ankitpanda008,Ankit Kumar Panda,Data Science enthusiast.,0,1,no data,no url,no social account has been linked,3,0.0,0,"CV,Projects-Data-Analysis,linkedin-skill-asses...","https://www.github.com/ankitpanda008/CV,https:...",No Repository link,No Repository link
1,MohanPrathap,No username,No bio,0,2,no data,no url,no social account has been linked,0,0.0,0,No Repository,No Repository link,No Repository link,No Repository link
2,imayank45,Mayank Kathane,No bio,0,1,no data,no url,no social account has been linked,22,0.0,3,"Data-Analysis,SQL-MySQL,GDP-Analysis,Stock-Dat...",https://www.github.com/imayank45/Data-Analysis...,"imayank45 / Data-Preprocessing,imayank45 / OS-...",https://www.github.com/imayank45/Data-Preproce...
3,ChidambaraRaju,Chidambara Raju,A Data Alchemist who Transforms Raw Data into ...,4,6,no data,https://www.kaggle.com/rajugc,Kaggle,14,0.0,34,"Web-Scraping-Project-IPL-Dataset,Youtube-Data-...",https://www.github.com/ChidambaraRaju/Web-Scra...,ChidambaraRaju / Web-Scraping-Project-IPL-Data...,https://www.github.com/ChidambaraRaju/Web-Scra...
4,prashant-3200,No username,No bio,0,1,no data,no url,no social account has been linked,0,0.0,0,No Repository,No Repository link,No Repository link,No Repository link


# Saving the data (Final)

In [34]:
followers_data.to_csv('followers_data.csv',index = False)